In [4]:
import math
import numpy as np

class MorphologyInCIELab:
    def __init__(self):
        pass

    def opening(self, image, structuring_element):
        return np.minimum(image, self.dilate(image, structuring_element))

    def dilate(self, image, structuring_element):
        return np.maximum(image, self.convolve(image, structuring_element))

    def convolve(self, image, kernel):
        kernel_height, kernel_width = kernel.shape
        padded_image = np.pad(image, ((kernel_height // 2, kernel_height // 2),
                                       (kernel_width // 2, kernel_width // 2)), mode='constant')
        result = np.zeros_like(image)
        for i in range(image.shape[0]):
            for j in range(image.shape[1]):
                result[i, j] = np.sum(padded_image[i:i+kernel_height, j:j+kernel_width] * kernel)
        return result

def remove_hair(image):
    morphology = MorphologyInCIELab()

    # Define structuring element for hair removal
    hair_removal_se = bank_of_structuring_elements(side_enclosing_square_in_px=25, num_orientations=12)

    hair_removal_as_list = []
    for idx, se in enumerate(hair_removal_se):
        hair_removal_as_list.append(morphology.opening(image, structuring_element=se))

    hair_removal = np.stack(hair_removal_as_list, axis=2)  # Hair removal results are stacked on the 3rd dimension.

    # Compute hair mask
    hair_mask = np.any(hair_removal < 0.9, axis=2)

    # Inpainting
    inpainted_image = inpaint(image, hair_mask)

    return inpainted_image


def bank_of_structuring_elements(side_enclosing_square_in_px, num_orientations):
    return [se_bar(side_enclosing_square_in_px=side_enclosing_square_in_px, orientation_in_degrees=a)
            for a in np.linspace(start=0, stop=180, num=num_orientations, endpoint=False)]


def se_bar(side_enclosing_square_in_px, orientation_in_degrees):
    se_sz = side_enclosing_square_in_px
    sz_ct = side_enclosing_square_in_px // 2
    m = -math.tan(math.radians(orientation_in_degrees))
    [coord_x, coord_y] = np.meshgrid(range(-sz_ct, se_sz - sz_ct), range(-sz_ct, se_sz - sz_ct))

    if m > 1e15:
        distance_to_line = np.abs(coord_x)
    else:
        distance_to_line = np.abs(m * coord_x - coord_y) / math.sqrt(m ** 2 + 1)

    variance = max(1 / 2, se_sz / 14)
    structuring_element = np.exp(-distance_to_line ** 2 / (2 * variance))
    return structuring_element


def inpaint(image, mask):
    # Replace masked areas with average color of nearby pixels
    inpainted_image = image.copy()
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            if mask[i, j]:
                nearby_pixels = []
                for di in range(-1, 2):
                    for dj in range(-1, 2):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < image.shape[0] and 0 <= nj < image.shape[1]:
                            if not mask[ni, nj]:
                                nearby_pixels.append(image[ni, nj])
                if nearby_pixels:
                    inpainted_image[i, j] = np.mean(nearby_pixels, axis=0)
    return inpainted_image


from skimage import io

# Load the image from file path
image_path = "/content/ISIC_0025363.webp"
image = io.imread(image_path)

# Call the remove_hair function
inpainted_image, steps = remove_hair(image)

# Now you can use inpainted_image for further processing or visualization



ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (2,2)  and requested shape (3,2)